In [1]:
#load the data
import pandas as pd

trends = pd.read_csv("../data/raw/google_trends_istanbul.csv", parse_dates=["date"])
weather = pd.read_csv("../data/processed/weather_istanbul_weekly.csv", parse_dates=["date"])
trends.head()
weather.head()

,date,temp_max,temp_min,temp_avg,precipitation
0,2021-01-03,14.566667,7.266667,10.733333,4.4
1,2021-01-10,14.314286,8.571429,11.471429,40.1
2,2021-01-17,9.000000,3.657143,6.071429,63.8
3,2021-01-24,8.671429,3.028571,5.885714,2.7
4,2021-01-31,10.885714,4.957143,7.785714,38.5


In [2]:
#aggregate trends so all categories have one demand score
trend_weekly = trends.groupby("date")["trend_score"].mean().reset_index()
trend_weekly.rename(columns={"trend_score": "trend_demand"}, inplace=True)

In [3]:
#then merged the data with weather
data = pd.merge(trend_weekly, weather, on="date", how="inner")
data.head()

,date,trend_demand,temp_max,temp_min,temp_avg,precipitation
0,2021-01-17,6.4,9.000000,3.657143,6.071429,63.8
1,2021-01-24,0.0,8.671429,3.028571,5.885714,2.7
2,2021-01-31,0.0,10.885714,4.957143,7.785714,38.5
3,2021-02-07,0.0,14.957143,7.385714,10.814286,4.8
4,2021-02-14,0.0,12.614286,7.228571,9.142857,42.4


In [4]:
%pip install scikit-learn

  Using cached scikit_learn-1.8.0-cp313-cp313-macosx_10_13_x86_64.whl.metadata (11 kB)
  Using cached scipy-1.17.0-cp313-cp313-macosx_10_14_x86_64.whl.metadata (62 kB)
  Using cached joblib-1.5.3-py3-none-any.whl.metadata (5.5 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.8.0-cp313-cp313-macosx_10_13_x86_64.whl (8.5 MB)
Using cached joblib-1.5.3-py3-none-any.whl (309 kB)
Using cached scipy-1.17.0-cp313-cp313-macosx_10_14_x86_64.whl (31.3 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)

[notice] A new release of pip is available: 24.2 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#normalize variables
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

data[["trend_demand", "temp_avg", "precipitation"]] = scaler.fit_transform(
    data[["trend_demand", "temp_avg", "precipitation"]]
)

In [6]:
#crowd indexx formual
data["crowd_index"] = (
    0.5 * data["trend_demand"] +
    0.3 * data["temp_avg"] +
    0.2 * (1 - data["precipitation"])
)

In [7]:
#catergories to help us visualize
data["crowd_level"] = pd.qcut(
    data["crowd_index"],
    q=3,
    labels=["Low", "Medium", "High"]
)

In [8]:
data.to_csv("../data/processed/weekly_crowd_index.csv", index=False)
data.head()

,date,trend_demand,temp_max,temp_min,temp_avg,precipitation,crowd_index,crowd_level
0,2021-01-17,0.197531,9.000000,3.657143,0.156558,0.440000,0.257733,Low
1,2021-01-24,0.000000,8.671429,3.028571,0.149819,0.018621,0.241221,Low
2,2021-01-31,0.000000,10.885714,4.957143,0.218766,0.265517,0.212526,Low
3,2021-02-07,0.000000,14.957143,7.385714,0.328668,0.033103,0.291980,Low
4,2021-02-14,0.000000,12.614286,7.228571,0.268015,0.292414,0.221922,Low


In [10]:
data.describe()

,date,trend_demand,temp_max,temp_min,temp_avg,precipitation,crowd_index
count,260,260.000000,260.000000,260.000000,260.000000,260.000000,260.000000
mean,2023-07-12 12:00:00,0.192521,19.420586,11.969963,0.497225,0.099106,0.425607
min,2021-01-17 00:00:00,0.000000,3.671429,-1.000000,0.000000,0.000000,0.145655
25%,2022-04-15 06:00:00,0.000000,13.017857,6.828571,0.282659,0.010172,0.320411
50%,2023-07-12 12:00:00,0.206790,19.457143,11.057143,0.474857,0.056897,0.407485
75%,2024-10-07 18:00:00,0.302469,26.585714,17.771429,0.731985,0.154138,0.539515
max,2026-01-04 00:00:00,1.000000,35.357143,23.928571,1.000000,1.000000,0.926518
std,NaN,0.179865,7.632759,6.556086,0.253583,0.120361,0.141544
